In [1]:
%%capture
import mrcnn
import mrcnn.config
import mrcnn.model
import mrcnn.visualize
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%%capture
CLASS_NAMES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

class SimpleConfig(mrcnn.config.Config):
    NAME = "coco_inference"
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    NUM_CLASSES = len(CLASS_NAMES)

model = mrcnn.model.MaskRCNN(mode="inference", 
                             config=SimpleConfig(),
                             model_dir=os.getcwd())

model.load_weights("dataset/mask_rcnn_coco.h5", by_name=True)

2022-04-28 14:28:07.038358: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-04-28 14:28:12.478234: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-28 14:28:12.478314: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kabilan-pt4987): /proc/driver/nvidia/version does not exist
2022-04-28 14:28:12.480073: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-28 14:28:12.541032: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2599990000 Hz
2022-04-28 14:28:12.542740: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a40ab258d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-28

In [3]:
def process(im_path,file_name):
    image = cv2.imread(im_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    r = model.detect([image], verbose=0)[0]
    im = cv2.imread(im_path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    for each in r['rois']:
      im = cv2.rectangle(im,(each[1],each[0]),(each[3],each[2]),(0,0,0),-1)
    im = cv2.cvtColor(im,cv2.COLOR_RGB2BGR)
    cv2.imwrite("pdataset/background/"+file_name,im)

#     im = cv2.imread(im_path)
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#     mask = np.zeros(im.shape[:2], dtype="uint8")
#     for each in r['rois']:
#       cv2.rectangle(mask,(each[1],each[0]),(each[3],each[2]),255,-1)
#     masked = cv2.bitwise_and(im, im, mask=mask)
#     masked = cv2.cvtColor(masked,cv2.COLOR_RGB2BGR)
#     cv2.imwrite("pdataset/objects/"+file_name,masked)

In [4]:
for each in os.listdir('mdataset/images'):
    process('mdataset/images/'+each,each)

2022-04-28 14:29:18.973062: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 67108864 exceeds 10% of system memory.
2022-04-28 14:29:19.043326: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 67108864 exceeds 10% of system memory.
2022-04-28 14:29:19.081366: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 67108864 exceeds 10% of system memory.
